<font size=7 face="Courier"> Perceptron Source Code

This is the code used to create the graphs in the perceptron notebook.

**Code References**
* [jupyter widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* [arrow documentaiton](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.arrow.html) and [arrow tutorial](https://www.geeksforgeeks.org/matplotlib-pyplot-arrow-in-python/)
* [legend tutorial](https://matplotlib.org/3.5.0/tutorials/intermediate/legend_guide.html)

## Import stuff

In [1]:
# Basic Packages
from math import cos, sin, pi
from sklearn.datasets import load_iris   # Package to laod data
import sklearn
from matplotlib import pyplot as plt     # Common plotting package
from matplotlib.patches import Patch, Circle     # Package to add colors in legend
import numpy as np                       # Common package for working wit data
import random

# Jupyte Widgets
from ipywidgets import interact          # Package for building GUI
import ipywidgets as ipw
from IPython.display import HTML, display, Javascript, clear_output

## Perceptron Class

This is the simplest example of the perceptron algorithm that contains a history of previous events for the algorithm.

In [2]:
class PerceptronSimple:
    """
    This code implements the perceptron algorithim for a single perceptron.... EXPLAINE MORE 
    Notes:
    * THe input,x, is a vector whose first value should always be 1
    * w*x can either be compared against a true output, y, or against the true weights, u
    """
    
    def __init__(self, w ):
        """
        We initialize weights and create arrays to hold previous data values for future analysis
        We initializing the class the user must decide whether to compare x against true output, y, or true weights, u
        """
        self.w = w
        self.data = type('data', (object,), {})()
        self.data.w      =  np.empty((0,len(w)))             # Create arrays to hold history parameter values
        self.data.x      =  np.empty((0,len(w)))
        self.data.y      =  np.empty((0))
        self.data.y_pred =  np.empty((0))
        
    def predict(self, x ):
        "Gives a prediction for y, using x and the weights 'w' inside the perceptron"
        return -1. if np.dot(self.w, x)<0 else 1.
    
    def update( self, x, y):     
        "Updates the weights 'w' based on the correctness of the perceptron's prediction. Logs previous values to history"
        self.data.w = np.append( self.data.w, [self.w], 0 )      # Save old values fror w, x, and y
        self.data.x = np.append( self.data.x, [x], 0 )
        self.data.y = np.append( self.data.y, y )                
        y_pred = self.predict(x)                                 # Get models prediction for y
        self.data.y_pred = np.append( self.data.y_pred, y_pred ) # Save models prediction for y 
        
        self.w = self.w + (y!=y_pred) * float(y)*np.array(x)     # Update weights based on x,y,y_pred
        
        
    def getHistory(self):
        "Returns history of all previous x,y,y_pred,w values from previous time steps"
        return vars(self.data)
                                  
    def getW(self):
        "Returns the valeu of the current weights 'w' . Weights can also be received by running 'object_name.w' "
        return self.w


## guiLinearClassifier Example

This function is used to so that students can try drawing a boundary between a set of datapoints.

In [12]:
def guiLinearClassifier():
    @interact( slope=(-2.0, 2.0, .5), intercept=(-2, 3, 0.5) )  # Creates an interactive GUI
    def f(slope, intercept):  
        x = np.linspace(0, 5, num=1000)       # Create dummy points for interactive line
        plt.rcParams["figure.figsize"]=8,5              # Set size of plot
        plt.ylim(-.2, 1.9)                    # Set Y-Axis
        print(f"Petal_Width = {slope} Petal_Length + {intercept}")
        plt.plot(x, slope * x + intercept, c="g")         # Plot interactive line

        iris = load_iris()                    # Load iris data
        x = iris.data[ iris.target <2, 2: ]   # Set Inputs data, X, to petal width and petal length
        y = iris.target[ iris.target < 2 ]    # Get outputs for tw ospecies of flowers

        plt.scatter( x[:,0], x[:,1], c=y, cmap="bwr")         # Create scatter plot of data
        plt.xlabel(iris.feature_names[2])                     # Add x and y axis labels
        plt.ylabel(iris.feature_names[3]) 
        legend_elements = [Patch(facecolor="r"), Patch(facecolor="b") ]   # Create colors in legend
        plt.legend(legend_elements, ["Tulips","Roses"])       # Add legend
        plt.show()                                            # Show plot

## Perceptron example

Here we show a visualization of the perceptron happening

In [5]:
def guiPerceptron():
    # Create Dataset
    x = np.array([[-.7,.2],[.5,.5],[.7,.3],[-.3,.6],[0,.7],[-.5,-.5],[.8,-.2],[.1,-.8],[-.7,-.3],[-.3,-.4]])  
    y = np.array([-1,-1,-1,-1,-1,1,1,1,1,1])     #y = np.array([1,1,1,1,1,-1,-1,-1,-1,-1]) # alternate example
    np.random.seed(1)
    ites= np.random.choice(10,10,replace=False)
    x,y = x[ites]*5, y[ites]
    
    @interact( step=(0, 10, 1) )  # Creates an interactive GUI
    def f(step):  
        # Scatterplot of datapoints
        plt.rcParams["figure.figsize"]=8,8              # Set size of plot
        plt.axhline(0, color="gray", linewidth=.5)
        plt.axvline(0, color='gray', linewidth=.5)
        colormap = np.array(['dummy', 'orange', 'blue'])
        plt.scatter( x[:,0], x[:,1], c=colormap[y]) #c=colormap[y])#, cmap=colormap[y])         # Create scatter plot of data
        plt.legend( [Patch(facecolor="b"),Patch(facecolor="orange")], ["$-1$","$1$"], loc="lower right")       # Add legend
        plt.axis([-5, 5, -5, 5])                             # Set x and y axis

        # set up perceptron  and step through updates
        w = [-1,-1]
        learner= PerceptronSimple(w)
        for j in range(step):
            i = j % len(y)                  # Cycle through datapoints over and over again
            learner.update(x[i],y[i])       # Draw weights vector and correpsonding Linear Classifier

        # Show the last point that was updated
        if step==0:
            display(ipw.HTMLMath("<h4>The <span class='text-success'>weight vector</span> is initialized to $w=[-1,-1]$</h4>") )
        elif step>0 and y[i]==learner.data.y_pred[i]:
            display(ipw.HTMLMath(f"<h4>The selected point $x=[{x[i,0]},{x[i,1]}]$ was <span class='text-success'>correctly classified</span>. No update occurs.</h4>") )
            plt.scatter( x[i,0], x[i,1], s=300 , facecolors="none", edgecolors="g", linewidth=2 )
        elif step>0 and y[i]>learner.data.y_pred[i]:
            display(ipw.HTMLMath(f"<h4>The selected point $x=[{x[i,0]},{x[i,1]}]$ was <span class='text-danger'>incorrecttly classified</span>. Since $y=$<font color='orange'>$1$</font>, we have "+"$w_t=w_{t-1}+x$</h4>") )
            plt.scatter( x[i,0], x[i,1], s=300 , facecolors="none", edgecolors="r", linewidth=2 )
            plt.arrow( 0,0, x[i,0],x[i,1], facecolor="r",alpha=.5, width = 0.05, length_includes_head=True, edgecolor="None")
            plt.arrow(0, 0, learner.data.w[-1,0],learner.data.w[-1,1], facecolor="g",alpha=.5, width = 0.05, length_includes_head=True, edgecolor="None")  
            plt.arrow( learner.data.w[-1,0],learner.data.w[-1,1], x[i,0],x[i,1], facecolor="r",alpha=.5, width = 0.05, length_includes_head=True, edgecolor="None")  
        elif step>0 and y[i]<learner.data.y_pred[i]:
            display(ipw.HTMLMath(f"<h4>The selected point $x=[{x[i,0]},{x[i,1]}]$ was <span class='text-danger'>incorrecttly classified</span>. Since $y=$<font color='blue'>$-1$</font>, we have "+"$w_t=w_{t-1}-x$</h4>") )
            plt.scatter( x[i,0], x[i,1], s=300 , facecolors="none", edgecolors="r", linewidth=2 )
            plt.arrow( 0,0, x[i,0],x[i,1], facecolor="r",alpha=.5, width = 0.05, length_includes_head=True, edgecolor="None")
            plt.arrow(0, 0, learner.data.w[-1,0],learner.data.w[-1,1], facecolor="g",alpha=.5, width = 0.05, length_includes_head=True, edgecolor="None")  
            plt.arrow( learner.data.w[-1,0],learner.data.w[-1,1], -x[i,0],-x[i,1], facecolor="r",alpha=.5, width = 0.05, length_includes_head=True, edgecolor="None")     
            
        # check which are classified positive
        for j in range(len(x)):
            if learner.predict(x[j]) != y[j]:
                plt.scatter( x[j,0], x[j,1],  s=220 ,facecolors="none", edgecolors="r", linewidth=.3 ) 
        plt.arrow(0,0, learner.w[0],learner.w[1], facecolor="green", width=0.1, length_includes_head=True, edgecolor="None") 
        plt.plot( [-10,10], -learner.w[0]/learner.w[1] * np.array([-10,10]), c="black", linewidth=1)    # Linear classifier line

## Weights2D

This example shows how weights create a linear classifier boundary

In [6]:
def guiWeights2D():

    # Set up graph appearance
    plt.rcParams["figure.figsize"]=6,6                    # Set size of plot
    plt.axhline(0, color="black", linewidth=.7)
    plt.axvline(0, color='black', linewidth=.7)
    plt.grid(linestyle = '--', linewidth = 0.3)
    plt.axis([-4, 4, -4, 4])                             # Set x and y axis
    plt.xlabel("$x_1$", fontsize=16)  
    plt.ylabel("$x_2$", fontsize=16) 
    
    # Add data, arrows, and lines
    plt.scatter( [-3,-3,2,2], [-2.5,1,-3,3], c=[-1,1,-1,1], cmap="bwr" ) 
    plt.arrow(0,0, 1, -2, facecolor="green", width=0.1, length_includes_head=True, edgecolor="None") 
    plt.arrow(0,0, 2, 1, facecolor="orange", width=0.1, alpha=.5, length_includes_head=True, edgecolor="None") 
    plt.plot( [-10,10], 1/2 * np.array([-10,10]), c="black", alpha=.6, linewidth=1)    # Linear classifier line
    plt.annotate("$w=[1,-2]$", (1.1,-1.9))
    plt.annotate("$x=[2,1]$", (2.1,.8))
    plt.show()

## guiBias2D

This shows a 2D graph that is only lineary separable if there is a bias term.

In [14]:
def guiBias2D():
    x = [-1,-1,1,1]
    y = [.5,1.5,.5,1.5]
    c=[-1,1,-1,1]

    plt.axhline(0, color="gray", linewidth=.5)
    plt.axhline(1, color="green", linewidth=.5)   # linear classifier
    plt.axvline(0, color='gray', linewidth=.5)
    plt.xlabel("$x_1$", fontsize=16)  
    plt.ylabel("$x_2$", fontsize=16) 
    plt.scatter( x, y, c=c, cmap="bwr" )         # Create scatter plot of data
    plt.legend( [Patch(facecolor="b"),Patch(facecolor="r")], ["$-1$","$1$"], loc="lower right")       # Add legend
    plt.axis([-2, 2, -1, 2])                             # Set x and y axis
    plt.show()

## guiBias3D

This shows a 3D graph that is only lineary separable if there is a bias term.

In [8]:
def guiBias3D():
    # set data
    x = [-1,-1,1,1]
    y = [.5,1.5,.5,1.5]
    z = [1,1,1,1]
    c=[-1,1,-1,1]
    
    # Set up graph and draw datapoints
    plt.rcParams['figure.figsize'] = (6,6)
    ax = plt.axes(projection='3d');
    ax.scatter3D(x, y, z ,  c=c, cmap="bwr");
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.view_init(elev=5, azim=-13)

    # Draw Origin
    x1, y1, z1 = np.array([[-2,0,0],[0,-2,0],[0,0,-2]])
    u, v, w = np.array([[4,0,0],[0,4,0],[0,0,4]])
    ax.quiver(x1,y1,z1,u,v,w,arrow_length_ratio=0.1, color="black")
    ax.grid(False)
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_zlabel('$x_3$')
    ax.set_xlim(-2,2)
    ax.set_ylim(-2,2)
    ax.set_zlim(-2,2)

    # plot the surface
    xx, yy = np.meshgrid( np.linspace(-2,2,50), np.linspace(-2,2,50) )
    zz = ( yy) 
    ax.plot_surface(xx, yy, zz, alpha=0.15, color="green")

    # Plot weights vector
    ax.quiver(0,0,0,0,-1,1,arrow_length_ratio=0.1, color="green")

    plt.show()

## guiNormalize

Show that normalizing data does not affect the results of the perceptron algorithm

In [9]:
def guiNormalize():
    plt.rcParams["figure.figsize"]=15,7              # Set size of plot
    # Create Initial Data
    x = np.array([[-.7,.2],[.5,.5],[.7,.3],[-.3,.6],[0,.7],[-.5,-.5],[.8,-.2],[.1,-.8],[-.7,-.3],[-.3,-.4]])  
    x = x * 2
    y = np.array([-1,-1,-1,-1,-1,1,1,1,1,1])     #y = np.array([1,1,1,1,1,-1,-1,-1,-1,-1]) # alternate example
    
    # Creat sublot framework
    fig, (plt1, plt2) = plt.subplots(1, 2)
    
    # Create First Plot Draw Scatter of initial Data
    plt1.axhline(0, color="gray", linewidth=.5)
    plt1.axvline(0, color='gray', linewidth=.5)
    colormap = np.array(['dummy', 'orange', 'blue'])
    plt1.scatter( x[:,0], x[:,1], c=colormap[y]) #c=colormap[y])#, cmap=colormap[y])         # Create scatter plot of data
    plt1.legend( [Patch(facecolor="b"),Patch(facecolor="orange")], ["$-1$","$1$"], loc="lower right")       # Add legend
    plt1.set_title("Unnormalized")
    plt1.axis([-2, 2, -2, 2]) 
    #plt.axis([-1, 1, -1, 1])    
    
    # Draw data initial weights for  w=[ 0.5 -4.5]
    w=np.array([ 0.5, -4.5])/3
    plt1.arrow(0, 0, w[0], w[1], facecolor="g",alpha=.5, width = 0.02, length_includes_head=True, edgecolor="None") 
    plt1.plot( [-10,10], -w[0]/w[1] * np.array([-10,10]), c="black", linewidth=1)    # Linear classifier line
    
    # Normalize data and draw scatter
    x_norm = sklearn.preprocessing.normalize(x)
    w_norm = sklearn.preprocessing.normalize(w.reshape(1, -1))[0]
    for i in x_norm:
        plt1.arrow(0, 0, i[0], i[1], facecolor="yellow",alpha=.5, width = 0.02, length_includes_head=True, edgecolor="None") 
    
    # Create Second Plot
    # Create data points
    plt2.axhline(0, color="gray", linewidth=.5)
    plt2.axvline(0, color='gray', linewidth=.5)
    colormap = np.array(['dummy', 'orange', 'blue'])
    plt2.scatter( x_norm[:,0], x_norm[:,1], c=colormap[y]) #c=colormap[y])#, cmap=colormap[y])         # Create scatter plot of data
    plt2.legend( [Patch(facecolor="b"),Patch(facecolor="orange")], ["$-1$","$1$"], loc="lower right")       # Add legend
    plt2.axis([-2, 2, -2, 2])  #axis([-1.1, 1.1, -1.1, 1.1])  
    plt2.set_title("Normalized")
    
    # Create Arrows
    for i in x_norm:
        plt2.arrow(0, 0, i[0], i[1], facecolor="yellow",alpha=.5, width = 0.01, length_includes_head=True, edgecolor="None") 
    plt2.arrow(0, 0, w_norm[0], w_norm[1], facecolor="g",alpha=.5, width = 0.02, length_includes_head=True, edgecolor="None") 
    plt2.plot( [-10,10], -w[0]/w[1] * np.array([-10,10]), c="black", linewidth=1)    # Linear classifier line